In [ ]:
# pip install konlpy

In [ ]:
# pip install scikit-learn

In [ ]:
# pip install gensim scikit-learn

In [ ]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [ ]:
# 필요한 라이브러리를 임포트
import os
from xml.etree import ElementTree
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
import os
from xml.etree import ElementTree
import json

# 폴더 내의 모든 파일 경로를 가져오는 함수
def get_file_paths(directory):
    file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))]
    return file_paths

def extract_bsisFacts_from_folder(folder_path):
    bsisFacts_list = []  # 'bsisFacts' 값을 저장할 리스트

    # 지정된 폴더 내의 모든 파일 목록을 얻기 위해 os.listdir()를 사용
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)  # 파일 경로 생성
        if os.path.isfile(file_path) and filename.endswith('.json'):
            # 파일인지 확인하고 확장자가 '.json'인 파일만 처리
            with open(file_path, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if 'facts' in content and 'bsisFacts' in content['facts']:
                    # 'bsisFacts'가 'facts' 안에 존재하는 경우
                    bsisFacts = content['facts']['bsisFacts']
                    bsisFacts_list.extend(bsisFacts)  # 리스트 확장

    return bsisFacts_list


# 폴더 경로 지정
civil_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/civil/'  # 민사 사건 파일이 있는 폴더 경로
criminal_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/criminal/'  # 형사 사건 파일이 있는 폴더 경로

# 데이터 로드
civil_texts = extract_bsisFacts_from_folder(civil_case_directory)
criminal_texts = extract_bsisFacts_from_folder(criminal_case_directory)

# 민사 및 형사 텍스트 데이터를 하나의 데이터셋으로 결합
texts = civil_texts + criminal_texts  # 전체 텍스트 데이터
labels = ['민사'] * len(civil_texts) + ['형사'] * len(criminal_texts)  # 민사: 0, 형사: 1

from konlpy.tag import Okt

# Okt 객체를 생성합니다.
okt = Okt()

# 사용자 정의 불용어 리스트를 정의합니다.
stop_words = ['는', '의', '에', '을', '를', '과', '와', '이', '가', '도', '으로', '로', '에서', '하다', '형사', '민사', '형법', '민법']

# 민사 및 형사 텍스트가 저장된 변수를 정의합니다.
# civil_texts = ["민사 예시 텍스트 데이터", "민사 텍스트의 또 다른 예시"]
# criminal_texts = ["형사 예시 텍스트 데이터", "형사 텍스트의 또 다른 예시"]

# 텍스트를 토큰화합니다.
civil_tokens = [okt.morphs(text) for text in civil_texts]
criminal_tokens = [okt.morphs(text) for text in criminal_texts]

# 불용어를 제거합니다.
civil_processed = [[word for word in tokens if word not in stop_words] for tokens in civil_tokens]
criminal_processed = [[word for word in tokens if word not in stop_words] for tokens in criminal_tokens]

# 각 리스트의 단어들을 집합으로 변환합니다.
# civil_word_set = set(word for sublist in civil_processed for word in sublist)
# criminal_word_set = set(word for sublist in criminal_processed for word in sublist)

# 두 집합의 교집합(공통 단어)을 찾습니다.
# common_words = civil_word_set.intersection(criminal_word_set)

# 공통 단어를 제거합니다.
# civil_final = [[word for word in sublist if word not in common_words] for sublist in civil_processed]
# criminal_final = [[word for word in sublist if word not in common_words] for sublist in criminal_processed]

# 결과를 변수에 저장합니다.
civil_texts = [' '.join(tokens) for tokens in civil_processed]
criminal_texts = [' '.join(tokens) for tokens in criminal_processed]

In [ ]:
print(len(civil_texts))

454


In [ ]:
print(civil_texts)

['심 “ 송수 ” “ 송수 ” “ 배후 조정 ” “ 배후 조종 ” 고친다', '심 소외 원고 원고 민주화 명예 회복 심의 심의 민주화 명예 회복 6123 호로 제정 13289 호로 되기 전의 민주화', '소외 28,092,120원 원고 원고 45,617,000원', '심 "[ 근거 ]" 환송 민주화 명예 회복 심의 조회', '민사소송법 420조 심판 인용', '원고 태양광 발전 태양광 발전 적합한 가능한지 피고', '부란 가축 사육제 구역', '원고 태양광 발전 소외 474,660,000원 소외 50,000,000원', '원고 문화재 현상 구역 태양광 발전', '원고 50,000,000원', '적용 문화재 현상 구역', '홈페이지 홈페이지 홈페이지 뉴스 『 취재 커지는 입시 비리 의혹 … 폭풍 예고 』 『 취재 플러스 커지는 야구 입시 비리 의혹 』 사무국 원고 업무방해 고소했다', '원고 야구 어기 강압 고교생 했다는 게재', '피고 취재', '변론 게재 홈페이지 뉴스 서비스 였고 웹 에이전시 피고', '피고 뉴스 서비스 웹 에이전시 수반 포괄 진흥 9조 뉴스 서비스', '게재 홈페이지 뉴스 서비스 콘텐츠 허브 였고 웹 에이전시 피고 피고 뉴스 서비스 웹 에이전시 수반 포괄 뉴스 서비스', '투수 이닝 투구 타자 타석 한하여 강화하였다', '원고 소외 사무국 고교야구 소외 소외 왕중 왕', '소외 소외 소외 왕중 왕전 이닝 투구 하였기에 출전', '원고 “ 원하는 원하는 ” 들며 소외 소외 왕중 왕전', '파악 원고 업무방해 강요 고소하였다', '탈 외적 효력 원고 불기소처분 불기소처분 재정신청 재정신청 항고 기각', '피고 원고 야구 어기 강압 했고 그렇게 고교생 합격 원고 업무방해 고소했다', '9월 5일 11일 소외 소외', '왕중 왕전', '요강 따르면 투수 이닝 타자 타석 들어서야', '소외 소외 충족', '소외 소외 성공하였다는 적시하였다', '피고 경마 경마 오고 겨울철 경마장 경주로 트랙 모래 얼어서 경마 지장 생기는 방지 200 t 다량 소

In [ ]:
# 필요한 라이브러리를 불러옵니다.
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torch
from sklearn.model_selection import train_test_split
from konlpy.tag import Okt
import os
import json

In [ ]:
# device 설정 (CUDA가 사용 가능한 경우 GPU 사용, 그렇지 않으면 CPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




In [ ]:
# ... 여기에 파일 로딩 및 전처리 코드를 삽입 ...

# 전처리된 텍스트 데이터와 레이블을 결합합니다.
texts = civil_texts + criminal_texts  # 전체 텍스트 데이터
labels = [0] * len(civil_texts) + [1] * len(criminal_texts)  # 민사: 0, 형사: 1

# BERT 토크나이저를 로드합니다.
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

# BERT에 맞게 데이터를 인코딩합니다.
encoded_data = tokenizer.batch_encode_plus(
    texts,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']
labels = torch.tensor(labels)

# 데이터셋을 생성합니다.
dataset = TensorDataset(input_ids, attention_masks, labels)

# 데이터를 훈련 세트와 검증 세트로 분리합니다.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# 데이터 로더를 생성합니다.
batch_size = 4

train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset),
    batch_size = batch_size
)

validation_dataloader = DataLoader(
    val_dataset,
    sampler = SequentialSampler(val_dataset),
    batch_size = batch_size
)

# BERT 분류 모델을 로드합니다.
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# 옵티마이저를 설정합니다.
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# 훈련 루프를 정의합니다.
epochs = 3
# 모델을 device로 보냅니다.
model.to(device)
for epoch in range(0, epochs):
    # 훈련 단계
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        model.zero_grad()
        outputs = model(**inputs)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    # 평가 단계
    model.eval()
    for batch in validation_dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        with torch.no_grad():
            outputs = model(**inputs)
            loss = outputs[0]
            logits = outputs[1]
        # 여기서 logits를 사용하여 정확도 등을 계산할 수 있습니다.


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ber

In [ ]:
from sklearn.metrics import accuracy_score

# 훈련 루프를 정의합니다.
epochs = 3
for epoch in range(epochs):
    # 훈련 단계
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        model.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    # 평가 단계
    model.eval()
    val_loss = 0
    predictions, true_labels = [], []
    for batch in validation_dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs.loss
        logits = outputs.logits
        val_loss += loss.item()

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].to('cpu').numpy()

        # 예측 값 저장
        predictions.append(logits)
        true_labels.append(label_ids)

    # 평가 결과 계산
    predictions = np.concatenate(predictions, axis=0)
    true_labels = np.concatenate(true_labels, axis=0)
    predicted_labels = np.argmax(predictions, axis=1)
    accuracy = accuracy_score(true_labels, predicted_labels)

    print(f'Epoch {epoch + 1}/{epochs}')
    print(f'Training loss: {total_loss / len(train_dataloader)}')
    print(f'Validation loss: {val_loss / len(validation_dataloader)}')
    print(f'Accuracy: {accuracy}')


Epoch 1/3
Training loss: 0.023832691336448643
Validation loss: 0.05388624414037032
Accuracy: 0.9797297297297297
Epoch 2/3
Training loss: 0.024252557956552245
Validation loss: 0.08928470295772108
Accuracy: 0.9695945945945946
Epoch 3/3
Training loss: 0.015681076579337084
Validation loss: 0.07445759098265774
Accuracy: 0.972972972972973


In [ ]:
import os
import json
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification

# 폴더 내의 모든 파일 경로를 가져오는 함수
def get_file_paths(directory):
    file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))]
    return file_paths

# 'bsisFacts' 값을 추출하는 함수
def extract_bsisFacts_from_folder(folder_path):
    bsisFacts_list = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith('.json'):
            with open(file_path, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if 'facts' in content and 'bsisFacts' in content['facts']:
                    bsisFacts = content['facts']['bsisFacts']
                    bsisFacts_list.extend(bsisFacts)
    return bsisFacts_list

# 불용어 처리를 수행하는 함수
def remove_stopwords(text, stop_words):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# 폴더 경로 지정
civil_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/civil/'  # 민사 사건 파일이 있는 폴더 경로
criminal_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/criminal/'  # 형사 사건 파일이 있는 폴더 경로

# 데이터 로드 및 전처리
civil_texts = extract_bsisFacts_from_folder(civil_case_directory)
criminal_texts = extract_bsisFacts_from_folder(criminal_case_directory)

# 사용자 정의 불용어 리스트 정의
stop_words = ['는', '의', '에', '을', '를', '과', '와', '이', '가', '도', '으로', '로', '에서', '하다', '형사', '민사', '형법', '민법']

# 불용어 제거
civil_texts = [remove_stopwords(text, stop_words) for text in civil_texts]
criminal_texts = [remove_stopwords(text, stop_words) for text in criminal_texts]

texts = civil_texts + criminal_texts  # 전체 텍스트 데이터
labels = ['민사'] * len(civil_texts) + ['형사'] * len(criminal_texts)  # 민사: 0, 형사: 1

# BERT 토크나이저와 모델 로드
model_name = 'bert-base-multilingual-cased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 파일에서 텍스트 데이터를 불러오고 전처리
new_example_file_path = '/content/drive/MyDrive/SKT_FLY_AI/Project/86가합493.json'
pred_texts = []
with open(new_example_file_path, 'r', encoding='utf-8') as file:
    new_example_text = json.load(file)
    if 'facts' in new_example_text:
        # 'facts' 부분만 문자열로 변환하여 추가
        facts_text = json.dumps(new_example_text['facts'], ensure_ascii=False)
        pred_texts.append(remove_stopwords(facts_text, stop_words))

# BERT 토크나이저를 사용하여 데이터를 인코딩
encoded_data = tokenizer.batch_encode_plus(
    pred_texts,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids = encoded_data['input_ids'].to(device)
attention_mask = encoded_data['attention_mask'].to(device)

# 모델을 사용하여 예측
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# 예측된 레이블을 가져옴
predicted_label = torch.argmax(logits, dim=1).cpu().numpy()[0]

# 예측된 분류 결과 출력
class_names = ['민사', '형사']
print("예측된 분류:", class_names[predicted_label])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


예측된 분류: 민사


In [ ]:
import os
import json
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification

# 폴더 내의 모든 파일 경로를 가져오는 함수
def get_file_paths(directory):
    file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))]
    return file_paths

# 'bsisFacts' 값을 추출하는 함수
def extract_bsisFacts_from_folder(folder_path):
    bsisFacts_list = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith('.json'):
            with open(file_path, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if 'facts' in content and 'bsisFacts' in content['facts']:
                    bsisFacts = content['facts']['bsisFacts']
                    bsisFacts_list.extend(bsisFacts)
    return bsisFacts_list

# 불용어 처리를 수행하는 함수
def remove_stopwords(text, stop_words):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# 폴더 경로 지정
civil_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/civil/'  # 민사 사건 파일이 있는 폴더 경로
criminal_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/criminal/'  # 형사 사건 파일이 있는 폴더 경로

# 데이터 로드 및 전처리
civil_texts = extract_bsisFacts_from_folder(civil_case_directory)
criminal_texts = extract_bsisFacts_from_folder(criminal_case_directory)

# 사용자 정의 불용어 리스트 정의
stop_words = ['는', '의', '에', '을', '를', '과', '와', '이', '가', '도', '으로', '로', '에서', '하다', '형사', '민사', '형법', '민법']

# 불용어 제거
civil_texts = [remove_stopwords(text, stop_words) for text in civil_texts]
criminal_texts = [remove_stopwords(text, stop_words) for text in criminal_texts]

texts = civil_texts + criminal_texts  # 전체 텍스트 데이터
labels = ['민사'] * len(civil_texts) + ['형사'] * len(criminal_texts)  # 민사: 0, 형사: 1

# BERT 토크나이저와 모델 로드
model_name = 'bert-base-multilingual-cased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 파일에서 텍스트 데이터를 불러오고 전처리
new_example_file_path = '/content/drive/MyDrive/SKT_FLY_AI/Project/2010고단3085.json'
pred_texts = []
with open(new_example_file_path, 'r', encoding='utf-8') as file:
    new_example_text = json.load(file)
    if 'facts' in new_example_text:
        # 'facts' 부분만 문자열로 변환하여 추가
        facts_text = json.dumps(new_example_text['facts'], ensure_ascii=False)
        pred_texts.append(remove_stopwords(facts_text, stop_words))

# BERT 토크나이저를 사용하여 데이터를 인코딩
encoded_data = tokenizer.batch_encode_plus(
    pred_texts,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids = encoded_data['input_ids'].to(device)
attention_mask = encoded_data['attention_mask'].to(device)

# 모델을 사용하여 예측
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# 예측된 레이블을 가져옴
predicted_label = torch.argmax(logits, dim=1).cpu().numpy()[0]

# 예측된 분류 결과 출력
class_names = ['민사', '형사']
print("예측된 분류:", class_names[predicted_label])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


예측된 분류: 형사


In [ ]:
import os
import json
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification

# 폴더 내의 모든 파일 경로를 가져오는 함수
def get_file_paths(directory):
    file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))]
    return file_paths

# 'bsisFacts' 값을 추출하는 함수
def extract_bsisFacts_from_folder(folder_path):
    bsisFacts_list = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith('.json'):
            with open(file_path, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if 'facts' in content and 'bsisFacts' in content['facts']:
                    bsisFacts = content['facts']['bsisFacts']
                    bsisFacts_list.extend(bsisFacts)
    return bsisFacts_list

# 불용어 처리를 수행하는 함수
def remove_stopwords(text, stop_words):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# 폴더 경로 지정
civil_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/civil/'  # 민사 사건 파일이 있는 폴더 경로
criminal_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/criminal/'  # 형사 사건 파일이 있는 폴더 경로

# 데이터 로드 및 전처리
civil_texts = extract_bsisFacts_from_folder(civil_case_directory)
criminal_texts = extract_bsisFacts_from_folder(criminal_case_directory)

# 사용자 정의 불용어 리스트 정의
stop_words = ['는', '의', '에', '을', '를', '과', '와', '이', '가', '도', '으로', '로', '에서', '하다', '형사', '민사', '형법', '민법']

# 불용어 제거
civil_texts = [remove_stopwords(text, stop_words) for text in civil_texts]
criminal_texts = [remove_stopwords(text, stop_words) for text in criminal_texts]

texts = civil_texts + criminal_texts  # 전체 텍스트 데이터
labels = ['민사'] * len(civil_texts) + ['형사'] * len(criminal_texts)  # 민사: 0, 형사: 1

# BERT 토크나이저와 모델 로드
model_name = 'bert-base-multilingual-cased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 파일에서 텍스트 데이터를 불러오고 전처리
new_example_file_path = '/content/drive/MyDrive/SKT_FLY_AI/Project/98가단28503.json'
pred_texts = []
with open(new_example_file_path, 'r', encoding='utf-8') as file:
    new_example_text = json.load(file)
    if 'facts' in new_example_text:
        # 'facts' 부분만 문자열로 변환하여 추가
        facts_text = json.dumps(new_example_text['facts'], ensure_ascii=False)
        pred_texts.append(remove_stopwords(facts_text, stop_words))

# BERT 토크나이저를 사용하여 데이터를 인코딩
encoded_data = tokenizer.batch_encode_plus(
    pred_texts,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids = encoded_data['input_ids'].to(device)
attention_mask = encoded_data['attention_mask'].to(device)

# 모델을 사용하여 예측
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# 예측된 레이블을 가져옴
predicted_label = torch.argmax(logits, dim=1).cpu().numpy()[0]

# 예측된 분류 결과 출력
class_names = ['민사', '형사']
print("예측된 분류:", class_names[predicted_label])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


예측된 분류: 민사


In [ ]:
# 잘못 나온 예시

import os
import json
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification

# 폴더 내의 모든 파일 경로를 가져오는 함수
def get_file_paths(directory):
    file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))]
    return file_paths

# 'bsisFacts' 값을 추출하는 함수
def extract_bsisFacts_from_folder(folder_path):
    bsisFacts_list = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith('.json'):
            with open(file_path, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if 'facts' in content and 'bsisFacts' in content['facts']:
                    bsisFacts = content['facts']['bsisFacts']
                    bsisFacts_list.extend(bsisFacts)
    return bsisFacts_list

# 불용어 처리를 수행하는 함수
def remove_stopwords(text, stop_words):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# 폴더 경로 지정
civil_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/civil/'  # 민사 사건 파일이 있는 폴더 경로
criminal_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/criminal/'  # 형사 사건 파일이 있는 폴더 경로

# 데이터 로드 및 전처리
civil_texts = extract_bsisFacts_from_folder(civil_case_directory)
criminal_texts = extract_bsisFacts_from_folder(criminal_case_directory)

# 사용자 정의 불용어 리스트 정의
stop_words = ['는', '의', '에', '을', '를', '과', '와', '이', '가', '도', '으로', '로', '에서', '하다', '형사', '민사', '형법', '민법']

# 불용어 제거
civil_texts = [remove_stopwords(text, stop_words) for text in civil_texts]
criminal_texts = [remove_stopwords(text, stop_words) for text in criminal_texts]

texts = civil_texts + criminal_texts  # 전체 텍스트 데이터
labels = ['민사'] * len(civil_texts) + ['형사'] * len(criminal_texts)  # 민사: 0, 형사: 1

# BERT 토크나이저와 모델 로드
model_name = 'bert-base-multilingual-cased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 파일에서 텍스트 데이터를 불러오고 전처리
new_example_file_path = '/content/drive/MyDrive/SKT_FLY_AI/Project/98나4361.json'
pred_texts = []
with open(new_example_file_path, 'r', encoding='utf-8') as file:
    new_example_text = json.load(file)
    if 'facts' in new_example_text:
        # 'facts' 부분만 문자열로 변환하여 추가
        facts_text = json.dumps(new_example_text['facts'], ensure_ascii=False)
        pred_texts.append(remove_stopwords(facts_text, stop_words))

# BERT 토크나이저를 사용하여 데이터를 인코딩
encoded_data = tokenizer.batch_encode_plus(
    pred_texts,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids = encoded_data['input_ids'].to(device)
attention_mask = encoded_data['attention_mask'].to(device)

# 모델을 사용하여 예측
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# 예측된 레이블을 가져옴
predicted_label = torch.argmax(logits, dim=1).cpu().numpy()[0]

# 예측된 분류 결과 출력
class_names = ['민사', '형사']
print("예측된 분류:", class_names[predicted_label])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


예측된 분류: 형사


In [ ]:
import os
import json
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification

# 폴더 내의 모든 파일 경로를 가져오는 함수
def get_file_paths(directory):
    file_paths = [os.path.join(directory, filename) for filename in os.listdir(directory) if os.path.isfile(os.path.join(directory, filename))]
    return file_paths

# 'bsisFacts' 값을 추출하는 함수
def extract_bsisFacts_from_folder(folder_path):
    bsisFacts_list = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith('.json'):
            with open(file_path, 'r', encoding='utf-8') as file:
                content = json.load(file)
                if 'facts' in content and 'bsisFacts' in content['facts']:
                    bsisFacts = content['facts']['bsisFacts']
                    bsisFacts_list.extend(bsisFacts)
    return bsisFacts_list

# 불용어 처리를 수행하는 함수
def remove_stopwords(text, stop_words):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# 폴더 경로 지정
civil_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/civil/'  # 민사 사건 파일이 있는 폴더 경로
criminal_case_directory = '/content/drive/MyDrive/SKT_FLY_AI/Project/criminal/'  # 형사 사건 파일이 있는 폴더 경로

# 데이터 로드 및 전처리
civil_texts = extract_bsisFacts_from_folder(civil_case_directory)
criminal_texts = extract_bsisFacts_from_folder(criminal_case_directory)

# 사용자 정의 불용어 리스트 정의
stop_words = ['는', '의', '에', '을', '를', '과', '와', '이', '가', '도', '으로', '로', '에서', '하다', '형사', '민사', '형법', '민법']

# 불용어 제거
civil_texts = [remove_stopwords(text, stop_words) for text in civil_texts]
criminal_texts = [remove_stopwords(text, stop_words) for text in criminal_texts]

texts = civil_texts + criminal_texts  # 전체 텍스트 데이터
labels = ['민사'] * len(civil_texts) + ['형사'] * len(criminal_texts)  # 민사: 0, 형사: 1

# BERT 토크나이저와 모델 로드
model_name = 'bert-base-multilingual-cased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 파일에서 텍스트 데이터를 불러오고 전처리
new_example_file_path = '/content/drive/MyDrive/SKT_FLY_AI/Project/2000고단7136.json'
pred_texts = []
with open(new_example_file_path, 'r', encoding='utf-8') as file:
    new_example_text = json.load(file)
    if 'facts' in new_example_text:
        # 'facts' 부분만 문자열로 변환하여 추가
        facts_text = json.dumps(new_example_text['facts'], ensure_ascii=False)
        pred_texts.append(remove_stopwords(facts_text, stop_words))

# BERT 토크나이저를 사용하여 데이터를 인코딩
encoded_data = tokenizer.batch_encode_plus(
    pred_texts,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids = encoded_data['input_ids'].to(device)
attention_mask = encoded_data['attention_mask'].to(device)

# 모델을 사용하여 예측
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits

# 예측된 레이블을 가져옴
predicted_label = torch.argmax(logits, dim=1).cpu().numpy()[0]

# 예측된 분류 결과 출력
class_names = ['민사', '형사']
print("예측된 분류:", class_names[predicted_label])

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


예측된 분류: 형사
